# 1 Why Synthetic Data Before Fine-Tuning

By now, the pattern should feel familiar: we don't escalate until the system tells us to.
Fine-tuning is no different. Before we change a model's weights, we need something to train on. And in most enterprise engagements, the training data simply does not exist in a usable form.

Customers have documents. They have institutional knowledge. They have years of accumulated expertise encoded in PDFs, wikis, and policy manuals. What they almost never have is a structured dataset of questions and high-quality answers that a model can learn from directly.

This is the gap that Synthetic Data Generation fills.

SDG is not about fabricating information. It is about transforming existing knowledge into a format that training pipelines can consume. We take the documents we have already ingested, chunked, and validated through retrieval, and we use them to generate question-answer pairs that reflect the domain. The documents contain the knowledge. SDG extracts the signal and reshapes it into something a model can actually learn from.

Why can't we just use the documents directly? Because training a model requires examples of the behavior you want. A 200-page rulebook is not an example of behavior. It is reference material. The model needs to see questions asked and answered correctly, repeatedly, across the full surface area of the domain. That is what SDG produces.
To do this in practice, we use sdg_hub, the Red Hat AI Innovation Team's open-source toolkit for building synthetic data generation pipelines. The framework is built around two core concepts: blocks and flows. Blocks are composable processing units, each responsible for a single transformation, such as generating a question from a document chunk, producing an answer, or evaluating faithfulness. Flows chain blocks together into complete pipelines defined in YAML. You describe the sequence of transformations declaratively, and the framework handles orchestration, validation, and execution.

In concrete terms, the workflow looks like this. You point a flow at your ingested documents. The flow discovers the dataset schema it needs, generates candidate question-answer pairs using a hosted LLM, and then runs built-in evaluation blocks that score each pair for faithfulness and relevancy. The output is a structured dataset of domain-specific Q&A pairs, each grounded in your source material and scored for quality. That dataset becomes the input for fine-tuning.

There is an important sequence dependency here. SDG quality is directly tied to everything we built earlier in this lab. The ingestion pipeline determines whether the source text is clean. The chunking strategy determines whether the generated pairs are coherent or fragmented. The retrieval layer determines whether we can validate that generated answers are actually grounded in real content. If any of those upstream stages are broken, SDG will faithfully reproduce the damage. It will generate confident, well-structured question-answer pairs that are subtly wrong, and you will carry that error forward into training.

This is why SDG comes after retrieval and evaluation, not before. It is not a shortcut. It is a manufacturing step that depends on the quality of every step that preceded it.

When the pipeline is sound, SDG gives us something powerful: enough structured, domain-specific training signal to make fine-tuning viable without requiring the customer to hand-label thousands of examples. That is a meaningful reduction in cost, time, and organizational friction.

But the order matters. Documents first. Ingestion second. Retrieval third. Evaluation fourth. And only then, when the system is stable and the failures are understood, do we generate the data that prepares us for model adaptation.
SDG is the bridge between "the system works well enough" and "the model needs to internalize this domain." It is not the destination. It is how we get there responsibly.

## 1.1 Install SDG Hub
Before we can generate anything, we need the toolkit. SDG Hub is a modular Python framework built by the Red Hat AI Innovation Team. It is open source, Apache 2.0 licensed, and designed specifically for building synthetic data generation pipelines using composable blocks and flows.

The core install pulls in the library itself. The examples extra adds the pre-built flows we will use in this section. These pre-built flows matter because they encode tested, validated generation pipelines that we can use out of the box rather than assembling one from scratch during a lab.

In [2]:
! pip install sdg-hub[examples] -q

Note: Earlier versions of the documentation reference a `[vllm]` extra. As of version 0.8.3, that extra has been removed. The core library already supports connecting to any OpenAI-compatible API endpoint, including vLLM, Ollama, and hosted services like the MaaS endpoint we configured in Section 2. No separate vLLM integration package is needed.
With the library installed, we can confirm that everything registered correctly by discovering the available flows and blocks. Flows are complete generation pipelines defined in YAML. Blocks are the individual processing units that flows chain together. Think of blocks as the atoms and flows as the molecules.

## 1.2 Import Libraries and Discover Available Flows

With the library installed, we can confirm that everything registered correctly by discovering the available flows and blocks. Flows are complete generation pipelines defined in YAML. Blocks are the individual processing units that flows chain together. Think of blocks as the atoms and flows as the molecules.

In [3]:
from sdg_hub.core.flow import FlowRegistry, Flow
from sdg_hub.core.blocks import BlockRegistry
from datasets import Dataset

# Auto-discover all registered flows and blocks
FlowRegistry.discover_flows()
BlockRegistry.discover_blocks()

# See what shipped with the install
print("Available flows:")
for name in FlowRegistry.list_flows():
    print(f"  - {name}")

[23:11:19] INFO     Discovered 7 flows                                                              ]8;id=735809;file:///opt/app-root/lib64/python3.12/site-packages/sdg_hub/core/flow/registry.py\registry.py]8;;\:]8;id=136869;file:///opt/app-root/lib64/python3.12/site-packages/sdg_hub/core/flow/registry.py#126\126]8;;\

┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ ID               ┃ Name                  ┃ Author                ┃ Tags                  ┃ Description          ┃
┡━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ clean-shadow-397 │ Advanced Japanese     │ SDG Hub Contributors  │ question-generation,  │ A comprehensive flow │
│                  │ Document Grounded     │                       │ knowledge-extraction, │ that generates       │
│                  │ Question-Answer       │                       │ qa-pairs,             │ high-quality         │
│                  │ Generation Flow for   │                       │ document-processing,  │ question-answer      │
│                  │ Knowledge Tuning      │                       │ educational,          │ pairs from Japanese  │
│                  │                       │                       │ multilingual,         │ input documents      │
│                  │                       │                       │ japanese              │ using multiple LLM   │
│                  │                       │                       │                       │ blocks for question  │
│                  │                       │                       │                       │ generation, answer   │
│                  │                       │                       │                       │ synthesis, and       │
│                  │                       │                       │                       │ quality evaluation.  │
│ epic-jade-656    │ Extractive Summary    │ SDG Hub Contributors  │ knowledge-tuning,     │ Generate extractive  │
│                  │ Knowledge Tuning      │                       │ document-internaliza… │ summary from the     │
│                  │ Dataset Generation    │                       │ question-generation,  │ input document. Each │
│                  │ Flow                  │                       │ knowledge-extractive… │ document is first    │
│                  │                       │                       │ qa-pairs,             │ converted into list  │
│                  │                       │                       │ extractive-summaries  │ of knowledge         │
│                  │                       │                       │                       │ segments for         │
│                  │                       │                       │                       │ creating extractive  │
│                  │                       │                       │                       │ summary and then     │
│                  │                       │                       │                       │ annotated with       │
│                  │                       │                       │                       │ context,             │
│                  │                       │                       │                       │ relationship and     │
│                  │                       │                       │                       │ relevance. This is   │
│                  │                       │                       │                       │ then converted into  │
│                  │                       │                       │                       │ Question-Answer      │
│                  │                       │                       │                       │ pairs.               │
│ green-clay-812   │ Structured Text       │ SDG Hub Contributors  │ text-analysis,        │ Multi-step pipeline  │
│                  │ Insights Extraction   │                       │ summarization, nlp,   │ for extracting       │
│                  │ Flow                  │                       │ structured-output,    │ structured insights  │
│                  │                       │                       │ insights,             │ from text including  │
│                  │                       │            

                                                 Available Blocks                                                  
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Block Name                  ┃ Category  ┃ Description                                                           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ AgentBlock                  │ agent     │ Execute agent frameworks (Langflow, etc.) on DataFrame rows           │
│ AgentResponseExtractorBlock │ agent     │ Extracts text content from agent framework responses                  │
│ ColumnValueFilterBlock      │ filtering │ Filters datasets based on column values using various comparison      │
│                             │           │ operations                                                            │
│ LLMChatBlock                │ llm       │ Unified LLM chat block supporting 100+ providers via LiteLLM          │
│ LLMResponseExtractorBlock   │ llm       │ Extracts specified fields from LLM response objects                   │
│ PromptBuilderBlock          │ llm       │ Formats prompts into structured chat messages or plain text using     │
│                             │           │ Jinja templates                                                       │
│ RegexParserBlock            │ parsing   │ Parses text content using regex patterns                              │
│ TagParserBlock              │ parsing   │ Parses text content using start/end tags                              │
│ TextParserBlock             │ parsing   │ DEPRECATED: Use TagParserBlock or RegexParserBlock                    │
│ DuplicateColumnsBlock       │ transform │ Duplicates existing columns with new names according to a mapping     │
│                             │           │ specification                                                         │
│ IndexBasedMapperBlock       │ transform │ Maps values from source columns to output columns based on choice     │
│                             │           │ columns using shared mapping                                          │
│ JSONStructureBlock          │ transform │ Combines multiple columns into a single column containing a           │
│                             │           │ structured JSON object                                                │
│ MeltColumnsBlock            │ transform │ Transforms wide dataset format into long format by melting columns    │
│                             │           │ into rows                                                             │
│ RenameColumnsBlock          │ transform │ Renames columns in a dataset according to a mapping specification     │
│ RowMultiplierBlock          │ transform │ Duplicates each row in the dataset a configurable number of times     │
│ SamplerBlock                │ transform │ Randomly samples n values from a list column and outputs to a new     │
│                             │           │ column                                                                │
│ TextConcatBlock             │ transform │ Combines multiple columns into a single column using a specified      │
│                             │           │ separator                                                             │
│ UniformColumnValueSetter    │ transform │ Replaces all values in a column with a single summary statistic       │
│                             │           │ (e.g., mode, mean, median)                                            │
└─────────────────────────────┴───────────┴───────────────────────────────────────────────────────────────────────┘

Summary: 18 blocks across 5 categories

Available flows:
  - {'id': 'loud-dawn-245', 'name': 'RAG Evaluation Dataset Flow'}
  - {'id': 'mild-thunder-748', 'name': 'Detailed Summary Knowledge Tuning Dataset Generation Flow'}
  - {'id': 'stellar-peak-605', 'name': 'Document Based Knowledge Tuning Dataset Generation Flow'}
  - {'id': 'epic-jade-656', 'name': 'Extractive Summary Knowledge Tuning Dataset Generation Flow'}
  - {'id': 'heavy-heart-77', 'name': 'Key Facts Knowledge Tuning Dataset Generation Flow'}
  - {'id': 'clean-shadow-397', 'name': 'Advanced Japanese Document Grounded Question-Answer Generation Flow for Knowledge Tuning'}
  - {'id': 'green-clay-812', 'name': 'Structured Text Insights Extraction Flow'}


You should see several pre-built flows in the output, including flows for question-answer generation, knowledge tuning, and reasoning data. We will be using one of these in the next step.

If the output is empty or the import fails, check that the install completed without errors and that your Python environment is 3.10 or newer.

## 1.3 Load a Pre-Built Q&A Generation Flow
Now that we can see what is available, we select the flow we will use for the rest of this section. We are using one of the pre-built flows that generates question-answer pairs from documents. This is the same pattern a customer would follow: take ingested documents, produce structured training data.



In [7]:
flow_name = "Advanced Document Grounded Question-Answer Generation Flow for Knowledge Tuning"
flow_path = FlowRegistry.get_flow_path(flow_name)
flow = Flow.from_yaml(flow_path)


/tmp/ipykernel_6538/862295401.py:2: DeprecationWarning: DeprecationWarning: The InstructLab Multi-Summary QA flow has been deprecated and removed. Consider using 'enhanced_summary_knowledge_tuning' instead.
  flow_path = FlowRegistry.get_flow_path(flow_name)


[23:17:55] WARNING  DeprecationWarning: The InstructLab Multi-Summary QA flow has been deprecated   ]8;id=59360;file:///opt/app-root/lib64/python3.12/site-packages/sdg_hub/core/flow/registry.py\registry.py]8;;\:]8;id=369989;file:///opt/app-root/lib64/python3.12/site-packages/sdg_hub/core/flow/registry.py#177\177]8;;\
                    and removed. Consider using 'enhanced_summary_knowledge_tuning' instead.                       

FlowValidationError: Flow path cannot be None. Please provide a valid YAML file path or check that the flow exists in the registry.

In [8]:
FlowRegistry.discover_flows()
flows = FlowRegistry.list_flows()
for f in flows:
    print(f)

┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ ID               ┃ Name                  ┃ Author                ┃ Tags                  ┃ Description          ┃
┡━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ clean-shadow-397 │ Advanced Japanese     │ SDG Hub Contributors  │ question-generation,  │ A comprehensive flow │
│                  │ Document Grounded     │                       │ knowledge-extraction, │ that generates       │
│                  │ Question-Answer       │                       │ qa-pairs,             │ high-quality         │
│                  │ Generation Flow for   │                       │ document-processing,  │ question-answer      │
│                  │ Knowledge Tuning      │                       │ educational,          │ pairs from Japanese  │
│                  │                       │                       │ multilingual,         │ input documents      │
│                  │                       │                       │ japanese              │ using multiple LLM   │
│                  │                       │                       │                       │ blocks for question  │
│                  │                       │                       │                       │ generation, answer   │
│                  │                       │                       │                       │ synthesis, and       │
│                  │                       │                       │                       │ quality evaluation.  │
│ epic-jade-656    │ Extractive Summary    │ SDG Hub Contributors  │ knowledge-tuning,     │ Generate extractive  │
│                  │ Knowledge Tuning      │                       │ document-internaliza… │ summary from the     │
│                  │ Dataset Generation    │                       │ question-generation,  │ input document. Each │
│                  │ Flow                  │                       │ knowledge-extractive… │ document is first    │
│                  │                       │                       │ qa-pairs,             │ converted into list  │
│                  │                       │                       │ extractive-summaries  │ of knowledge         │
│                  │                       │                       │                       │ segments for         │
│                  │                       │                       │                       │ creating extractive  │
│                  │                       │                       │                       │ summary and then     │
│                  │                       │                       │                       │ annotated with       │
│                  │                       │                       │                       │ context,             │
│                  │                       │                       │                       │ relationship and     │
│                  │                       │                       │                       │ relevance. This is   │
│                  │                       │                       │                       │ then converted into  │
│                  │                       │                       │                       │ Question-Answer      │
│                  │                       │                       │                       │ pairs.               │
│ green-clay-812   │ Structured Text       │ SDG Hub Contributors  │ text-analysis,        │ Multi-step pipeline  │
│                  │ Insights Extraction   │                       │ summarization, nlp,   │ for extracting       │
│                  │ Flow                  │                       │ structured-output,    │ structured insights  │
│                  │                       │                       │ insights,             │ from text including  │
│                  │                       │            

{'id': 'loud-dawn-245', 'name': 'RAG Evaluation Dataset Flow'}
{'id': 'mild-thunder-748', 'name': 'Detailed Summary Knowledge Tuning Dataset Generation Flow'}
{'id': 'stellar-peak-605', 'name': 'Document Based Knowledge Tuning Dataset Generation Flow'}
{'id': 'epic-jade-656', 'name': 'Extractive Summary Knowledge Tuning Dataset Generation Flow'}
{'id': 'heavy-heart-77', 'name': 'Key Facts Knowledge Tuning Dataset Generation Flow'}
{'id': 'clean-shadow-397', 'name': 'Advanced Japanese Document Grounded Question-Answer Generation Flow for Knowledge Tuning'}
{'id': 'green-clay-812', 'name': 'Structured Text Insights Extraction Flow'}


In [9]:
qa_flows = FlowRegistry.search_flows(tag="question-generation")
print(qa_flows)

[{'id': 'mild-thunder-748', 'name': 'Detailed Summary Knowledge Tuning Dataset Generation Flow'}, {'id': 'stellar-peak-605', 'name': 'Document Based Knowledge Tuning Dataset Generation Flow'}, {'id': 'epic-jade-656', 'name': 'Extractive Summary Knowledge Tuning Dataset Generation Flow'}, {'id': 'heavy-heart-77', 'name': 'Key Facts Knowledge Tuning Dataset Generation Flow'}, {'id': 'clean-shadow-397', 'name': 'Advanced Japanese Document Grounded Question-Answer Generation Flow for Knowledge Tuning'}]
